# Naive Bayes Klasifikasi Gaussian pada Cuaca (Biner & Categorical)

**Nama  : Rizki Ardian Samudra**

**NIM   : 210411100179**

**Kelas : Penambangan Data-C**

Naive Bayes pada dasarnya adalah sebuah algoritma klasifikasi yang berdasarkan pada teorema Bayes dengan mengasumsikan bahwa setiap fitur (atau atribut) pada data yang diamati adalah independen satu sama lain. Oleh karena itu, algoritma ini dianggap "naif" karena tidak mempertimbangkan interaksi antara fitur.

Dalam klasifikasi, Naive Bayes menghitung probabilitas setiap kelas yang mungkin berdasarkan pada nilai fitur yang diamati, dan kemudian memilih kelas dengan probabilitas tertinggi sebagai prediksi. Algoritma ini sering digunakan dalam analisis teks, seperti pengklasifikasian email spam atau klasifikasi dokumen berdasarkan topik, tetapi juga dapat diterapkan pada berbagai jenis masalah klasifikasi lainnya.

1. import library pandas, numpy, dan sklearn yang diperlukan

In [61]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

2. import data kita

In [62]:
file_url = 'https://drive.google.com/file/d/1zjQilkcbW9AlhC_NuRI-tkR2vPIYKV7G/view?usp=share_link'

file_id = file_url.split('/')[-2]
dwn_url = "https://drive.google.com/uc?id=" + file_id
adult_df = pd.read_csv(dwn_url, sep=';')
# adult_df.columns = ['outlook', 'humidity', 'windy', 'play', 'temperature']

jumlah_baris = adult_df.shape[0]
adult_df.head(jumlah_baris)

,outlook,humidity,windy,play,temperature
0,sunny,high,False,no,hot
1,sunny,high,True,no,hot
2,overcast,high,False,yes,hot
3,rainy,high,False,yes,mild
4,rainy,normal,False,yes,cool
5,rainy,normal,True,no,cool
6,sunny,high,False,no,mild
7,sunny,normal,False,yes,cool
8,rainy,normal,False,yes,mild
9,sunny,normal,True,yes,mild


3. Menangani data hilang atau missing value

> dibawah ini menghitung data kosong:

In [63]:
adult_df.isnull().sum()

outlook        0
humidity       0
windy          0
play           0
temperature    1
dtype: int64

> dibawah ini menghitung data `?`:

In [64]:
for value in ['outlook', 'humidity', 'windy', 'play', 'temperature']:
    print(value,":", sum(adult_df[value] == '?'))

outlook : 0
humidity : 0
windy : 0
play : 0
temperature : 0


4. Pemrosesan awal data atau prepocessing

In [65]:
adult_df_rev = adult_df  #duplikat data asli

5. Ringkasan statistik dari semua atribut:

In [66]:
adult_df_rev.describe(include= 'all')

,outlook,humidity,windy,play,temperature
count,14,14,14,14,13
unique,3,2,2,2,3
top,sunny,high,False,yes,mild
freq,5,7,8,9,6


6. Langkah Imputasi Data atau mengisi data hilang dengan nilai yang didasarkan top dari ringkasan statistik diatas

In [67]:
for value in ['outlook', 'humidity', 'windy', 'play', 'temperature']:
    # adult_df_rev[value].replace([None], [adult_df_rev.describe(include='all')[value][2]], inplace=True)  #mengisi berdasarkan top describe
    adult_df_rev[value].replace([None], ['cool'], inplace=True)  #mengisi dengan cool

adult_df_rev.head(jumlah_baris)

,outlook,humidity,windy,play,temperature
0,sunny,high,False,no,hot
1,sunny,high,True,no,hot
2,overcast,high,False,yes,hot
3,rainy,high,False,yes,mild
4,rainy,normal,False,yes,cool
5,rainy,normal,True,no,cool
6,sunny,high,False,no,mild
7,sunny,normal,False,yes,cool
8,rainy,normal,False,yes,mild
9,sunny,normal,True,yes,mild


7. One-Hot Encoder

> Encoder ini mengonversi data ke dalam format biner.

> Berikut adalah kode untuk menggunakan LabelEncoder pada beberapa kolom categorical pada dataset adult_df, dan kemudian melakukan encoding menggunakan One-Hot Encoder:

In [68]:
le = preprocessing.LabelEncoder()
outlook_cat = le.fit_transform(adult_df.outlook)
humidity_cat = le.fit_transform(adult_df.humidity)
windy_cat   = le.fit_transform(adult_df.windy)
play_cat = le.fit_transform(adult_df.play)
temperature_cat = le.fit_transform(adult_df.temperature)

In [69]:
#initialize the encoded categorical columns
adult_df_rev['outlook_cat'] = outlook_cat
adult_df_rev['humidity_cat'] = humidity_cat
adult_df_rev['windy_cat'] = windy_cat
adult_df_rev['play_cat'] = play_cat
adult_df_rev['temperature_cat'] = temperature_cat

In [70]:
#drop the old categorical columns from dataframe
dummy_fields = ['outlook', 'humidity', 'windy', 'play', 'temperature']
adult_df_rev = adult_df_rev.drop(dummy_fields, axis = 1)

adult_df_rev.head(jumlah_baris)

,outlook_cat,humidity_cat,windy_cat,play_cat,temperature_cat
0,2,0,0,0,1
1,2,0,1,0,1
2,0,0,0,1,1
3,1,0,0,1,2
4,1,1,0,1,0
5,1,1,1,0,0
6,2,0,0,0,2
7,2,1,0,1,0
8,1,1,0,1,2
9,2,1,1,1,2


8. Mengindeks ulang kolom menggunakan kode di bawah ini:

In [71]:
adult_df_rev = adult_df_rev.reindex(['outlook_cat', 'humidity_cat', 'windy_cat', 'play_cat', 'temperature_cat'], axis= 1)

adult_df_rev.head(1)

,outlook_cat,humidity_cat,windy_cat,play_cat,temperature_cat
0,2,0,0,0,1


9. Standardisasi Data atau Standardization of Data

> Semua nilai data dari kerangka data adalah numerik. Sekarang, kita perlu mengonversinya dalam satu skala. Kita bisa menstandarkan nilai. Kita dapat menggunakan rumus di bawah ini untuk standardisasi:

In [72]:
num_features = ['outlook_cat', 'humidity_cat', 'windy_cat', 'play_cat']

scaled_features = {}
for each in num_features:
    mean, std = adult_df_rev[each].mean(), adult_df_rev[each].std()
    scaled_features[each] = [mean, std]
    adult_df_rev.loc[:, each] = (adult_df_rev[each] - mean)/std

10. Pemotongan Data atau Data Slicing

> Pisahkan data menjadi data training and data test. Kita dapat dengan mudah melakukan langkah ini menggunakan metode train_test_split() dari sklearn:

> Variabel features_train berisi data fitur untuk training, features_test berisi data fitur untuk test, target_train berisi target untuk training, dan target_test berisi target untuk test.

In [73]:
features = adult_df_rev.values[:,:4]  #Variabel features diisi dengan nilai features dari dataset adult_df_rev dengan slicing [:,:4] artinya mengambil seluruh baris dan kolom 0 hingga kolom 3 (kolom ke-4 tidak diambil karena merupakan variabel target).
target = adult_df_rev.values[:,4]  #Variabel target diisi dengan nilai target dari dataset adult_df_rev dengan slicing [:,4] artinya mengambil seluruh baris dan hanya kolom ke-4 (variabel target).
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size = 0.33, random_state = 10)  #Kemudian menggunakan fungsi train_test_split, dataset yang sudah dipisahkan dijadikan argumen, dengan tambahan argumen test_size = 0.33 yang berarti dataset akan dibagi menjadi 33% data uji dan 67% data latih, dan random_state = 10 sebagai seed untuk menghasilkan hasil yang sama setiap kali dijalankan.

11. Implementasi Gaussian Naive Bayes

> Setelah selesai preprocessing data. saatnya menerapkan algoritma pembelajaran mesin atau machine learning. Kita akan menggunakan modul GaussianNB sklearn:

In [74]:
clf = GaussianNB()
clf.fit(features_train, target_train)
target_pred = clf.predict(features_test)

12. Akurasi model Gaussian Naive Bayes

In [75]:
accuracy = accuracy_score(target_test, target_pred, normalize = True)

print(f"Akurasi model KNN: {accuracy:.2%}")

Akurasi model KNN: 20.00%


## Kesimpulan

Berdasarkan nilai diatas tingkat akurasinya 20% . Ini menunjukkan data kita jelek. Kita dapat membuat kumpulan data uji acak dan menguji model untuk mengetahui seberapa baik kinerja model Gaussian Naive Bayes yang terlatih.